In [1]:
import json
import requests
import time
from xml.etree import ElementTree as ET

In [2]:
response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?id=13')
tree = ET.fromstring(response.content)

In [3]:
# List of all the elements
#[elem.tag for elem in tree.iter()]

In [4]:
# See the whole text in XML format
#print(ET.tostring(tree, encoding='utf8').decode('utf8'))

## Code to pull boardgame info

In [5]:
# Creating a function to get the boardgame data from each page

def boardgame_info(tree):
    
    id_num = int(tree[0].attrib['id'])
    boardgamecategory = []
    boardgamemechanic = []
    boardgamedesigner = []
    
    for i in range(len(tree[0])):
        
        if tree[0][i].tag == 'name':
            if tree[0][i].attrib['type'] == 'primary':
                boardgame_name = tree[0][i].attrib['value']
                
        elif tree[0][i].tag == 'yearpublished':
            year_publ = int(tree[0][i].attrib['value'])
            
        elif tree[0][i].tag == 'minplayers':
            minplayers = int(tree[0][i].attrib['value'])
            
        elif tree[0][i].tag == 'maxplayers':
            maxplayers = int(tree[0][i].attrib['value'])   
            
        elif tree[0][i].tag == 'playingtime':
            playingtime = int(tree[0][i].attrib['value']) 
            
        elif tree[0][i].tag == 'minplaytime':
            minplaytime = int(tree[0][i].attrib['value']) 
            
        elif tree[0][i].tag == 'maxplaytime':
            maxplaytime = int(tree[0][i].attrib['value'])
            
        elif tree[0][i].tag == 'link':
            
            if tree[0][i].attrib['type'] == 'boardgamecategory':
                boardgamecategory.append(tree[0][i].attrib['value'])
                
            elif tree[0][i].attrib['type'] == 'boardgamemechanic':
                boardgamemechanic.append(tree[0][i].attrib['value'])
                
            elif tree[0][i].attrib['type'] == 'boardgamedesigner':
                boardgamedesigner.append(tree[0][i].attrib['value'])
     
    keys = ['game ID#', 'boardgame', 'year published', 'min players', 'max players', 'playing time', 'min playing time', 'max playing time', 'category', 'boardgame mechanic', 'boardgamedesigner']
    values = [id_num, boardgame_name, year_publ, minplayers, maxplayers, playingtime, minplaytime, maxplaytime, boardgamecategory, boardgamemechanic, boardgamedesigner]
    dictionary = dict(zip(keys, values))

    return dictionary           

## Code to pull boardgame ratings

In [6]:
ratings_response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?id=13&ratingcomments=1&page=1')
ratings_tree = ET.fromstring(ratings_response.content)

In [7]:
# Creating a function to get the boardgame ratings

def get_ratings_per_page(ratings_tree):
        
    # Figuring out the number of ratings pages
    for i in range(len(ratings_tree[0])):
        if ratings_tree[0][i].tag == 'comments':
            ratings_info = i
            num_ratings = float(ratings_tree[0][i].attrib['totalitems'])
    #print(ratings_info)

    all_local_ratings = []
    for i in range(len(ratings_tree[0][ratings_info])):

        username = ratings_tree[0][ratings_info][i].attrib['username']
        rating = float(ratings_tree[0][ratings_info][i].attrib['rating'])     

        ratings_keys = ['username', 'rating']
        ratings_values = [username, rating]
        ratings_dictionary = dict(zip(ratings_keys, ratings_values))
        
        all_local_ratings.append(ratings_dictionary)
        
        #print(ratings_dictionary)
    return all_local_ratings

In [8]:
def get_all_ratings(ID):
    
    site = 'https://www.boardgamegeek.com/xmlapi2/thing?id='
    page_num = ID   
    ratings_site = '&ratingcomments=1&page='
    ratings_page_num_1 = 1
       
    ratings_website = site + str(page_num) + ratings_site + str(ratings_page_num_1)
    ratings_response = requests.get(ratings_website)
    ratings_tree = ET.fromstring(ratings_response.content)
        
   
    # Figuring out the number of ratings pages
    for j in range(len(ratings_tree[0])):
        if ratings_tree[0][j].tag == 'comments':
            num_ratings = float(ratings_tree[0][j].attrib['totalitems'])
    #print(num_ratings)
    
    num_pages = round(num_ratings/100)
    if num_pages < num_ratings/100:
        num_pages += 1
    #print(num_pages)

    all_ratings = []
    for k in range(num_pages):
        ratings_website2 = site + str(page_num) + ratings_site + str(k+1)
        ratings_response2 = requests.get(ratings_website2)
        ratings_tree2 = ET.fromstring(ratings_response2.content)
        #print(k, ratings_website2)
        all_ratings.append(get_ratings_per_page(ratings_tree2))
        time.sleep(2)        
    return all_ratings

## Pulling all info and ratings of multiple games

In [9]:
def get_all_game_info(ID):
    
    # boardgame websites
    site = 'https://www.boardgamegeek.com/xmlapi2/thing?id='
    page_num = ID
    website = site+str(page_num)        
    response = requests.get(website)
    tree = ET.fromstring(response.content)
       
    ratings_site = '&ratingcomments=1&page='
    ratings_page_num_1 = 1
       
    ratings_website = site + str(page_num) + ratings_site + str(ratings_page_num_1)
    ratings_response = requests.get(ratings_website)
    ratings_tree = ET.fromstring(ratings_response.content) 

    bg_info = boardgame_info(tree)
    bg_ratings = get_all_ratings(ID)
    bg_info['ratings'] = bg_ratings
    time.sleep(5)         

    
    return bg_info

In [11]:
for ID in range(3):
    site = 'https://www.boardgamegeek.com/xmlapi2/thing?id='
    page_num = ID
    website = site+str(page_num)        
    response = requests.get(website)
    tree = ET.fromstring(response.content)
    
    if len(tree) > 0 and tree[0].attrib['type'] == 'boardgame':
        ind_bg = get_all_game_info(ID)
        print(ID)
        with open('ind_bg_{0}.json'.format(ID), 'w') as json_file:
            json.dump('ind_bg_{0}.json'.format(ID), json_file)

1
2


In [12]:
with open('ind_bg_2.json') as f:
    ind_bg_2 = json.load(f)

In [13]:
print(json.dumps(ind_bg_2, indent=4, sort_keys=False,  separators=(',', ': ')))

"ind_bg_2.json"
